In [ ]:
pip install openai jupyter python-dotenv -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

load_dotenv(find_dotenv(), override=True)
client = OpenAI()

In [ ]:
def create_meals(
    ingredients,
    kcal=2000,
    exact_ingredients=False,
    output_format="text",
    model="gpt-3.5-turbo",
    system_role="Você é um cozinheiro habilidoso com a expertise de um chef.",
    temperature=1,
    extra=None,
):
    prompt = f"""
    Crie um plano diário de alimentação para café da manhã, almoço e janta baseado nos seguintes ingedientes: 
    {ingredients}
    Seu output tem ser no formato {output_format}
     Siga as instruções abaixo cuidadosamente
     ### Instruções:
        
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content